In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# --- 1. Initial Data Loading (as previously executed) ---
df = pd.read_csv('ckd_dataset.csv')

# --- 2. Data Preprocessing (as previously executed) ---
# Remove the first two rows (metadata)
df = df.iloc[2:].copy()
df.reset_index(drop=True, inplace=True)

# Clean column names
df.columns = df.columns.str.lower().str.replace(r'[^a-zA-Z0-9_]', '', regex=True).str.replace(' ', '_')

# Function to convert various string formats to numerical
def convert_to_numeric(value):
    if isinstance(value, (int, float)):
        return value
    if pd.isna(value) or value == 'nan' or value == '?':
        return np.nan
    try:
        if ' - ' in str(value):
            lower, upper = map(float, str(value).split(' - '))
            return (lower + upper) / 2
        elif str(value).startswith('< '):
            return float(str(value)[2:]) - 0.01
        elif str(value).startswith('≥ '):
            return float(str(value)[2:]) + 0.01
        else:
            return float(value)
    except ValueError:
        return value

# Apply the conversion function to all columns
for col in df.columns:
    df[col] = df[col].apply(convert_to_numeric)

# Convert 'class' column: 'ckd' to 1, 'notckd' to 0
df['class'] = df['class'].map({'ckd': 1, 'notckd': 0})

# Convert 'stage' column: 's1' to 1, 's2' to 2, ..., 's5' to 5
stage_mapping = {'s1': 1, 's2': 2, 's3': 3, 's4': 4, 's5': 5}
df['stage'] = df['stage'].map(stage_mapping)

# Re-attempt conversion for 'grf' column to numeric, coercing errors to NaN
df['grf'] = pd.to_numeric(df['grf'], errors='coerce')

# Impute missing value in 'grf' with the median
df['grf'].fillna(df['grf'].median(), inplace=True)

# Separate features (X) and target variables (y_ckd, y_stage)
X = df.drop(columns=['class', 'stage', 'affected'])
y_ckd = df['class']

# --- 3. CKD Presence Prediction Model Training and Evaluation ---
# Split data for CKD presence prediction
X_train_ckd, X_test_ckd, y_train_ckd, y_test_ckd = train_test_split(X, y_ckd, test_size=0.2, random_state=42)

# Initialize and train a RandomForestClassifier for CKD presence
ckd_model = RandomForestClassifier(n_estimators=100, random_state=42)
ckd_model.fit(X_train_ckd, y_train_ckd)

# Make predictions on the test set
y_pred_ckd = ckd_model.predict(X_test_ckd)

# Evaluate the model
print("--- CKD Presence Prediction Model Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test_ckd, y_pred_ckd):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_ckd, y_pred_ckd))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_ckd, y_pred_ckd))

C:\Users\saich\AppData\Local\Temp\ipykernel_24452\1117591783.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['grf'].fillna(df['grf'].median(), inplace=True)


--- CKD Presence Prediction Model Evaluation ---
Accuracy: 0.9750

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       0.96      1.00      0.98        25

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.98      0.97      0.97        40


Confusion Matrix:
[[14  1]
 [ 0 25]]


In [2]:
# --- 4. CKD Stage Prediction Model Training and Evaluation ---
# Filter the DataFrame for rows where CKD is present (class == 1)
df_ckd_present = df[df['class'] == 1].copy()

# Separate features (X_stage) and target (y_stage_filtered) for CKD stage prediction
X_stage = df_ckd_present.drop(columns=['class', 'stage', 'affected'])
y_stage_filtered = df_ckd_present['stage']

# Split data for CKD stage prediction (using stratified split due to potential class imbalance)
X_train_stage, X_test_stage, y_train_stage, y_test_stage = train_test_split(
    X_stage, y_stage_filtered, test_size=0.2, random_state=42, stratify=y_stage_filtered
)

# Initialize and train a RandomForestClassifier for CKD stage prediction
stage_model = RandomForestClassifier(n_estimators=100, random_state=42)
stage_model.fit(X_train_stage, y_train_stage)

# Make predictions on the test set for stage
y_pred_stage = stage_model.predict(X_test_stage)

# Evaluate the stage prediction model
print("\n--- CKD Stage Prediction Model Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test_stage, y_pred_stage):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_stage, y_pred_stage))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_stage, y_pred_stage))


--- CKD Stage Prediction Model Evaluation ---
Accuracy: 0.8077

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           2       1.00      0.33      0.50         3
           3       0.71      0.83      0.77         6
           4       0.78      0.88      0.82         8
           5       0.86      0.86      0.86         7

    accuracy                           0.81        26
   macro avg       0.87      0.78      0.79        26
weighted avg       0.83      0.81      0.80        26


Confusion Matrix:
[[2 0 0 0 0]
 [0 1 2 0 0]
 [0 0 5 1 0]
 [0 0 0 7 1]
 [0 0 0 1 6]]


In [3]:
# Filter the DataFrame for rows where CKD is present (class == 1)
df_ckd_present = df[df['class'] == 1].copy()

# Separate features (X_stage) and target (y_stage_filtered) for CKD stage prediction
X_stage = df_ckd_present.drop(columns=['class', 'stage', 'affected'])
y_stage_filtered = df_ckd_present['stage']

# Split data for CKD stage prediction (using stratified split due to potential class imbalance)
X_train_stage, X_test_stage, y_train_stage, y_test_stage = train_test_split(
    X_stage, y_stage_filtered, test_size=0.2, random_state=42, stratify=y_stage_filtered
)

# Initialize and train a RandomForestClassifier for CKD stage prediction
stage_model = RandomForestClassifier(n_estimators=100, random_state=42)
stage_model.fit(X_train_stage, y_train_stage)

# Make predictions and evaluate
# y_pred_stage = stage_model.predict(X_test_stage)
# print("\n--- CKD Stage Prediction Model Evaluation ---")
# print(f"Accuracy: {accuracy_score(y_test_stage, y_pred_stage):.4f}")
# print("\nClassification Report:\n", classification_report(y_test_stage, y_pred_stage))
# print("\nConfusion Matrix:\n", confusion_matrix(y_test_stage, y_pred_stage))

RandomForestClassifier(random_state=42)

In [31]:
from fpdf import FPDF
import datetime
import pandas as pd
import numpy as np

# ✅ Define preprocess_input
def preprocess_input(input_data):
    processed_input = pd.Series(input_data)

    def convert_to_numeric_single(value):
        if isinstance(value, (int, float)):
            return value
        if pd.isna(value) or value == 'nan' or value == '?':
            return np.nan
        try:
            if ' - ' in str(value):
                lower, upper = map(float, str(value).split(' - '))
                return (lower + upper) / 2
            elif str(value).startswith('< '):
                return float(str(value)[2:]) - 0.01
            elif str(value).startswith('≥ '):
                return float(str(value)[2:]) + 0.01
            else:
                return float(value)
        except ValueError:
            return value

    for col in processed_input.index:
        processed_input[col] = convert_to_numeric_single(processed_input[col])

    # Ensure columns match training features (X) in order and presence
    training_columns = X.columns  # ⚠️ Ensure X is defined in your notebook above this cell
    processed_df = pd.DataFrame([processed_input], columns=training_columns)

    # Impute 'grf' if missing in input using median from original data
    if 'grf' in processed_df.columns and processed_df['grf'].isnull().any():
        processed_df['grf'].fillna(df['grf'].median(), inplace=True)  # ⚠️ Ensure df is defined

    return processed_df

# ✅ Define prediction function
def predict_ckd_and_stage(input_data_dict):
    processed_input_df = preprocess_input(input_data_dict)
    ckd_prediction = ckd_model.predict(processed_input_df)[0]  # ⚠️ Ensure ckd_model is loaded above

    result = {
        'CKD_Presence': bool(ckd_prediction),
        'CKD_Stage': None
    }

    if ckd_prediction == 1:
        stage_prediction = stage_model.predict(processed_input_df)[0]  # ⚠️ Ensure stage_model is loaded above
        result['CKD_Stage'] = int(stage_prediction)

    return result

# ✅ Define PDF generation function
def generate_clinical_pdf(patient_name, age, gender, bp, sg, al, su, rbc, pc, pcc, ba, bgr, bu, sc, sodium, potassium, hemo, pcv, wc, rc, hypertension, diabetesmellitus, cad, appetite, peda, ane, prediction, stage):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt="Kidney Disease Clinical Report", ln=True, align='C')
    pdf.cell(200, 10, txt=f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)

    pdf.cell(200, 10, txt=f"Name: {patient_name}", ln=True)
    pdf.cell(200, 10, txt=f"Age: {age}", ln=True)
    pdf.cell(200, 10, txt=f"Gender: {gender}", ln=True)
    pdf.cell(200, 10, txt=f"Blood Pressure: {bp}", ln=True)
    pdf.cell(200, 10, txt=f"Specific Gravity: {sg}", ln=True)
    pdf.cell(200, 10, txt=f"Albumin: {al}", ln=True)
    pdf.cell(200, 10, txt=f"Sugar: {su}", ln=True)
    pdf.cell(200, 10, txt=f"Red Blood Cells: {rbc}", ln=True)
    pdf.cell(200, 10, txt=f"Pus Cell: {pc}", ln=True)
    pdf.cell(200, 10, txt=f"Pus Cell Clumps: {pcc}", ln=True)
    pdf.cell(200, 10, txt=f"Bacteria: {ba}", ln=True)
    pdf.cell(200, 10, txt=f"Blood Glucose Random: {bgr}", ln=True)
    pdf.cell(200, 10, txt=f"Blood Urea: {bu}", ln=True)
    pdf.cell(200, 10, txt=f"Serum Creatinine: {sc}", ln=True)
    pdf.cell(200, 10, txt=f"Sodium: {sodium}", ln=True)
    pdf.cell(200, 10, txt=f"Potassium: {potassium}", ln=True)
    pdf.cell(200, 10, txt=f"Hemoglobin: {hemo}", ln=True)
    pdf.cell(200, 10, txt=f"Packed Cell Volume: {pcv}", ln=True)
    pdf.cell(200, 10, txt=f"White Blood Cell Count: {wc}", ln=True)
    pdf.cell(200, 10, txt=f"Red Blood Cell Count: {rc}", ln=True)
    pdf.cell(200, 10, txt=f"Hypertension: {hypertension}", ln=True)
    pdf.cell(200, 10, txt=f"Diabetes Mellitus: {diabetesmellitus}", ln=True)
    pdf.cell(200, 10, txt=f"Coronary Artery Disease: {cad}", ln=True)
    pdf.cell(200, 10, txt=f"Appetite: {appetite}", ln=True)
    pdf.cell(200, 10, txt=f"Pedal Edema: {peda}", ln=True)
    pdf.cell(200, 10, txt=f"Anemia: {ane}", ln=True)

    pdf.cell(200, 10, txt=f"Prediction: {prediction}", ln=True)
    pdf.cell(200, 10, txt=f"CKD Stage: {stage}", ln=True)

    filename = f"{patient_name}_clinical_report.pdf"
    pdf.output(filename)
    print(f"✅ PDF generated successfully: {filename}")

# ✅ Sample usage
patient_name = input("Enter patient's name: ")
gender = input("Enter patient's gender: ")
age = hypothetical_patient_data['age']  # ⚠️ Ensure hypothetical_patient_data is defined above

# Make prediction
prediction_result = predict_ckd_and_stage(hypothetical_patient_data)

# Generate PDF
generate_clinical_pdf(
    patient_name=patient_name,
    age=age,
    gender=gender,
    bp=hypothetical_patient_data['bplimit'],
    sg=hypothetical_patient_data['sg'],
    al=hypothetical_patient_data['al'],
    su=hypothetical_patient_data['su'],
    rbc=hypothetical_patient_data['rbc'],
    pc=hypothetical_patient_data['pc'],
    pcc=hypothetical_patient_data['pcc'],
    ba=hypothetical_patient_data['ba'],
    bgr=hypothetical_patient_data['bgr'],
    bu=hypothetical_patient_data['bu'],
    sc=hypothetical_patient_data['sc'],
    sodium=hypothetical_patient_data['sod'],
    potassium=hypothetical_patient_data['pot'],
    hemo=hypothetical_patient_data['hemo'],
    pcv=hypothetical_patient_data['pcv'],
    wc=hypothetical_patient_data['wbcc'],
    rc=hypothetical_patient_data['rbcc'],
    hypertension=hypothetical_patient_data['htn'],
    diabetesmellitus=hypothetical_patient_data['dm'],
    cad=hypothetical_patient_data['cad'],
    appetite=hypothetical_patient_data['appet'],
    peda=hypothetical_patient_data['pe'],
    ane=hypothetical_patient_data['ane'],
    prediction="CKD Present" if prediction_result['CKD_Presence'] else "No CKD Detected",
    stage=prediction_result['CKD_Stage']
)


Enter patient's name:  jkl
Enter patient's gender:  female


✅ PDF generated successfully: jkl_clinical_report.pdf


C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:61: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:63: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Kidney Disease Clinical Report", ln=True, align='C')
C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:63: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Kidney Disease Clinical Report", ln=True, align='C')
C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:64: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt=f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)
C:\Users\saich\AppData\Local\Temp\ipyk

In [32]:
hypothetical_patient_data = {
    'bpdiastolic': 0.0,
    'bplimit': 0.0,
    'sg': 1.010,
    'al': 0.01,
    'rbc': 0.0,
    'su': -0.01,
    'pc': 0.0,
    'pcc': 0.0,
    'ba': 0.0,
    'bgr': 100.0,
    'bu': 30.0,
    'sod': 140.0,
    'sc': 1.0,
    'pot': 4.0,
    'hemo': 14.0,
    'pcv': 40.0,
    'rbcc': 5.0,
    'wbcc': 8000.0,
    'htn': 0.0,
    'dm': 0.0,
    'cad': 0.0,
    'appet': 1.0,
    'pe': 0.0,
    'ane': 0.0,
    'grf': 100.0,
    'age': 30.0
}

prediction_result = predict_ckd_and_stage(hypothetical_patient_data)
print(f"Prediction for Hypothetical Healthy Patient:")
print(f"  CKD Presence: {prediction_result['CKD_Presence']}")
print(f"  CKD Stage: {prediction_result['CKD_Stage']}")

Prediction for Hypothetical Healthy Patient:
  CKD Presence: True
  CKD Stage: 2


In [33]:
ckd_patient_data = {
    'bpdiastolic': 0.0,
    'bplimit': 0.0,
    'sg': 1.010,
    'al': 4.01,
    'rbc': 1.0,
    'su': -0.01,
    'pc': 1.0,
    'pcc': 0.0,
    'ba': 1.0,
    'bgr': 111.99,
    'bu': 67.15,
    'sod': 135.5,
    'sc': 3.64,
    'pot': 7.3,
    'hemo': 9.35,
    'pcv': 31.55,
    'rbcc': 4.755,
    'wbcc': 15690.0,
    'htn': 0.0,
    'dm': 0.0,
    'cad': 0.0,
    'appet': 1.0,
    'pe': 0.0,
    'ane': 0.0,
    'grf': 139.8635,
    'age': 11.99
}

prediction_result_ckd = predict_ckd_and_stage(ckd_patient_data)
print(f"\nPrediction for Hypothetical CKD Patient:")
print(f"  CKD Presence: {prediction_result_ckd['CKD_Presence']}")
print(f"  CKD Stage: {prediction_result_ckd['CKD_Stage']}")


Prediction for Hypothetical CKD Patient:
  CKD Presence: True
  CKD Stage: 1


In [34]:
# Input data for a patient who is likely healthy (CKD not present)
healthy_patient_input = {
    'bpdiastolic': 0.0,
    'bplimit': 0.0,
    'sg': 1.020,
    'al': -0.01,
    'rbc': 0.0,
    'su': -0.01,
    'pc': 0.0,
    'pcc': 0.0,
    'ba': 0.0,
    'bgr': 122.495,  # Blood Glucose Random
    'bu': 48.09,     # Blood Urea
    'sod': 140.5,    # Sodium
    'sc': 3.64,      # Serum Creatinine
    'pot': 7.3,      # Potassium
    'hemo': 14.55,   # Hemoglobin
    'pcv': 43.25,    # Packed Cell Volume
    'rbcc': 5.345,   # Red Blood Cell Count
    'wbcc': 8550.0,  # White Blood Cell Count
    'htn': 0.0,      # Hypertension (0 for No)
    'dm': 0.0,       # Diabetes Mellitus (0 for No)
    'cad': 0.0,      # Coronary Artery Disease (0 for No)
    'appet': 0.0,    # Appetite (0 for Good)
    'pe': 0.0,       # Pedal Edema (0 for No)
    'ane': 0.0,      # Anemia (0 for No)
    'grf': 114.698,  # Glomerular Filtration Rate
    'age': 47.0
}

# How to use it:
prediction_result = predict_ckd_and_stage(healthy_patient_input)
print(prediction_result)

{'CKD_Presence': False, 'CKD_Stage': None}


In [35]:
# Input data for a patient who is likely to have CKD
ckd_patient_input = {
    'bpdiastolic': 0.0,
    'bplimit': 0.0,
    'sg': 4.010,     # Specific Gravity (lower than healthy)
    'al': 5.01,      # Albumin (elevated)
    'rbc': 3.0,      # Red Blood Cells (presence)
    'su': -0.01,
    'pc': 1.0,       # Pus Cell (presence)
    'pcc': 2.0,
    'ba': 1.0,       # Bacteria (presence)
    'bgr': 111.99,
    'bu': 67.15,
    'sod': 135.5,
    'sc': 4.64,
    'pot': 8.3,
    'hemo': 9.35,    # Hemoglobin (low)
    'pcv': 34.55,    # Packed Cell Volume (low)
    'rbcc': 4.755,
    'wbcc': 15699.0, # White Blood Cell Count (elevated)
    'htn': 0.0,
    'dm': 0.0,
    'cad': 0.0,
    'appet': 1.0,
    'pe': 0.0,
    'ane': 0.0,
    'grf': 139.8635,
    'age': 55.99
}

# How to use it:
prediction_result_ckd = predict_ckd_and_stage(ckd_patient_input)
print(prediction_result_ckd)

{'CKD_Presence': True, 'CKD_Stage': 5}


In [36]:
# Make sure you have pandas, numpy, sklearn.model_selection, sklearn.ensemble imported
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Assuming df, X, y_ckd are already defined from your preprocessing steps
X_train_ckd, X_test_ckd, y_train_ckd, y_test_ckd = train_test_split(X, y_ckd, test_size=0.2, random_state=42)
ckd_model = RandomForestClassifier(n_estimators=100, random_state=42)
ckd_model.fit(X_train_ckd, y_train_ckd)
print("CKD Presence Model (ckd_model) retrained.")

CKD Presence Model (ckd_model) retrained.


In [37]:
# Make sure df_ckd_present, X_stage, y_stage_filtered are defined from filtering CKD positive patients
df_ckd_present = df[df['class'] == 1].copy()
X_stage = df_ckd_present.drop(columns=['class', 'stage', 'affected'])
y_stage_filtered = df_ckd_present['stage']

X_train_stage, X_test_stage, y_train_stage, y_test_stage = train_test_split(
    X_stage, y_stage_filtered, test_size=0.2, random_state=42, stratify=y_stage_filtered
)
stage_model = RandomForestClassifier(n_estimators=100, random_state=42)
stage_model.fit(X_train_stage, y_train_stage)
print("CKD Stage Model (stage_model) retrained.")

CKD Stage Model (stage_model) retrained.


In [16]:
import joblib
import os

models_dir = 'trained_models'
os.makedirs(models_dir, exist_ok=True)

ckd_model_path = os.path.join(models_dir, 'ckd_presence_model.joblib')
ckd_stage_model_path = os.path.join(models_dir, 'ckd_stage_model.joblib')

joblib.dump(ckd_model, ckd_model_path)
joblib.dump(stage_model, ckd_stage_model_path)
print("Models saved successfully after retraining.")

Models saved successfully after retraining.


In [13]:
!pip install --upgrade pip setuptools


  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.2 MB 3.1 MB/s eta 0:00:01
   ---------------- ----------------------- 0.5/1.2 MB 5.1 MB/s eta 0:00:01
   ------------------------- -------------- 0.8/1.2 MB 5.5 MB/s eta 0:00:01
   --------------------------------- ------ 1.0/1.2 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 5.9 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\saich\anaconda3\python.exe -m pip install --upgrade pip setuptools


In [15]:
!pip install fpdf2


     ---------------------------------------- 0.0/69.7 kB ? eta -:--:--
     ----------------------- ---------------- 41.0/69.7 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 69.7/69.7 kB 762.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/109.0 kB ? eta -:--:--
     ----------------------------------- -- 102.4/109.0 kB 3.0 MB/s eta 0:00:01
     -------------------------------------- 109.0/109.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/245.7 kB ? eta -:--:--
   ------------------------- -------------- 153.6/245.7 kB ? eta -:--:--
   ---------------------------------------- 245.7/245.7 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -- ------------------------------------- 0.1/2.2 MB 8.3 MB/s eta 0:00:01
   --- ------------------------------------ 0.2/2.2 MB 3.9 MB/s eta 0:00:01
   ----- ---------------------------------- 0.3/2.2 MB 2.4 MB/s eta 0:00:01
   ------- ---

In [21]:
import sys
print(sys.executable)


C:\Users\saich\anaconda3\envs\skl10\python.exe


In [22]:
!{sys.executable} -m pip install fpdf2


  Using cached fpdf2-2.8.3-py2.py3-none-any.whl.metadata (69 kB)
Using cached fpdf2-2.8.3-py2.py3-none-any.whl (245 kB)


In [23]:
from fpdf import FPDF


In [38]:
patient_name = input("Enter patient's name: ")
gender = input("Enter patient's gender: ")
age = hypothetical_patient_data['age']

generate_clinical_pdf(
    patient_name=patient_name,
    age=age,
    gender=gender,
    bp=hypothetical_patient_data['bplimit'],
    sg=hypothetical_patient_data['sg'],
    al=hypothetical_patient_data['al'],
    su=hypothetical_patient_data['su'],
    rbc=hypothetical_patient_data['rbc'],
    pc=hypothetical_patient_data['pc'],
    pcc=hypothetical_patient_data['pcc'],
    ba=hypothetical_patient_data['ba'],
    bgr=hypothetical_patient_data['bgr'],
    bu=hypothetical_patient_data['bu'],
    sc=hypothetical_patient_data['sc'],
    sodium=hypothetical_patient_data['sod'],
    potassium=hypothetical_patient_data['pot'],
    hemo=hypothetical_patient_data['hemo'],
    pcv=hypothetical_patient_data['pcv'],
    wc=hypothetical_patient_data['wbcc'],
    rc=hypothetical_patient_data['rbcc'],
    hypertension=hypothetical_patient_data['htn'],
    diabetesmellitus=hypothetical_patient_data['dm'],
    cad=hypothetical_patient_data['cad'],
    appetite=hypothetical_patient_data['appet'],
    peda=hypothetical_patient_data['pe'],
    ane=hypothetical_patient_data['ane'],
    prediction="CKD Present" if prediction_result['CKD_Presence'] else "No CKD Detected",
    stage=prediction_result['CKD_Stage']
)


Enter patient's name:  uy
Enter patient's gender:  male


✅ PDF generated successfully: uy_clinical_report.pdf


C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:61: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:63: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Kidney Disease Clinical Report", ln=True, align='C')
C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:63: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Kidney Disease Clinical Report", ln=True, align='C')
C:\Users\saich\AppData\Local\Temp\ipykernel_24452\289436968.py:64: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt=f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)
C:\Users\saich\AppData\Local\Temp\ipyk